# Demo 10 : Decision 🌳s



In this demo, you will implement:
- training a decision tree
- different methods for training a random forest

/

Dans ce labo, nous allons implémenter:

*   l'entrainement d'un arbre de décision
*   différentes méthodes pour entrainement une forêt aléatoire

## Prepare the data

We will use the Iris dataset again. Start by loading the data and splitting it into train/test sets.

/

## Preparation des données
Nous allons encore utiliser le dataset Iris. Commencez par charger les données et les séparer en ensemble d'entrainement/test

In [0]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets

iris = datasets.load_iris()

# Split into train/test
# Sépare en entrainement/test
from sklearn.model_selection import train_test_split

(iris_train_x, iris_test_x, 
 iris_train_y, iris_test_y) = train_test_split(iris.data, iris.target, test_size=100)

print(iris_train_x.shape)
print(iris_test_x.shape)

(50, 4)
(100, 4)


## Decision tree

To implement the basic decision tree, we will first implement some helper functions.

/

## Arbre de décision
Pour implémenter un arbre de décision basique, nous devons d'abord créer quelques fonctions utiles.

In [0]:
from collections import Counter

def get_majority_class(labels):
    # This function should return the most common label
    # in the input array.
    #
    # Cette fonction doit retourner le label le plus
    # fréquent dans le tableau d'entrée.
    pass

def compute_entropy(labels):
    # This function should compute the entropy
    # (= sum_l(-p_l log2 (p_l)) for each label l)
    # of the input array.
    #
    # Cette fonction doit retourner l'entropie
    # (= sum_l(-p_l log2 (p_l)) pour chaque label l)
    # du tableau d'entrée
    pass

example_labels = np.array([3,1,2,0,2])
print(get_majority_class(example_labels)) # should be 2
print(compute_entropy(example_labels)) # should be 1.9219280948873623

None
None


Let's write a **Node** class to use in the decision tree. A node in the tree can:
- ask a question (= determine whether examples have a certain feature greater than a certain threshold),
- find the best question to split a (subset of a) dataset, or
- if it is a leaf node, predict an output.

To find the best question for one feature, compute all the **midpoints** of that feature, and for each midpoint, compute the entropy of the resulting split when the question uses that midpoint. To find the best question overall, repeat this for all columns, and pick the best question.

For example, let's say you have this dataset:

```
array([[ 0.94, -1.45, -0.1 ,  1.26],
       [-0.4 ,  0.94,  0.69,  1.53],
       [-0.38, -0.53,  1.85, -0.89]])
```

The second column has the values `-1.45, 0.94, -0.53`. Sorting these values gives us `-1.45, -0.53,  0.94`. If we take the midpoints of these values, we get `-0.99, 0.21`.

/

Implémentons une classe **Node** (noeud) à utiliser dans l'arbre de décision. Un noeud dans l'arbre peut:

* poser une question (= determiner si les exemples ont un certain attribut plus grand qu'un certain seuil),
* trouver la meilleure question pour séparer un (sous-ensemble d'un) jeu de données, ou
* si c'est une feuille, faire une prédiction

Pour trouver la meilleure question pour un attribut, calculer tout les **valeurs intermédiaires** de cet attribut, et pour chaque valeur intermédiaire, calculer l'entropie de la séparation obtenue quand la question utilise cette valeur intermédiaire. Pour trouver la meilleur question, répeter ça pour chaque colonne, et choisir la meilleur question.

Par exemple, disons que vous ayez ce jeu de données:

```
array([[ 0.94, -1.45, -0.1 ,  1.26],
       [-0.4 ,  0.94,  0.69,  1.53],
       [-0.38, -0.53,  1.85, -0.89]])
```

La deuxième colonne a les valeurs `-1.45, 0.94, -0.53`. Trier ces valeurs nous donne `-1.45, -0.53, 0.94`. Si nous prenons les valeurs intermédiaires, nous obtenons `-0.99, 0.21`.

In [0]:
class Node():
    def __init__(self):
        self.threshold = None
        self.col = None
        self.is_leaf = None
        self.output_class = None
        self.left_child=None
        self.right_child=None
        
    def find_best_question(self, x, y):
        # x: np array of shape (number of examples, number of features)
        # y: np array of shape (number of examples,)
        #
        # x: tableau numpy de dimension (nombre d'exemples, nombre d'attributs)
        # y: tableau numpy de dimension (nombre d'exemples,)
        best_col = 0
        best_val = 0
        best_loss = np.inf
        
        num_cols = x.shape[1]
        valid_cols = np.arange(num_cols)
        for col in valid_cols:
            #
            # Compute the midpoints of this column's values here
            #
            # Calculer les valeurs intermédiaires de cette colonne ici
            #
            midpoints=[]
            for val in midpoints:
                #
                # Using col and val, split the labels
                # into left_labels, right_labels here
                #
                # En utilisant col et val, séparer les labels
                # en left_labels (labels_gauches) et right_labels
                # (labels_droits) ici
                #
                
                right_entropy = compute_entropy(right_labels)
                left_entropy = compute_entropy(left_labels)
                loss = left_entropy + right_entropy
                if right_labels.shape[0] == 0 or left_labels.shape[0] == 0:
                    continue
                
                if loss < best_loss:
                    best_loss = loss
                    best_col = col
                    best_val = val
                    
        self.col = best_col
        self.threshold = best_val
    
    def ask_question(self, x):
        if not self.is_leaf:
            return x[:, self.col] > self.threshold
        else:
            print("Error: leaf nodes cannot ask questions!")
            return False
    
    def predict(self):
        if self.is_leaf:
            return self.output_class
        else:
            print("Error: non-leaf nodes cannot make a prediction!")
            return None

node=Node()
example_x = np.array([[-2.32,  2.02,  0.53,  0.34],
       [-1.44,  1.36,  0.12, -0.44],
       [-0.28, -0.08,  0.9 , -1.63],
       [-0.09,  0.17, -0.28,  0.44],
       [ 0.8 , -1.65,  1.36,  1.62]])
example_y = np.array([3,1,2,0,2])

# Test find_best_question()
node.find_best_question(example_x, example_y)
print(node.threshold) # should be -1.88
print(node.col) # should be 0

# Test ask_question()
test_x = np.array([[ 1.05, -1.85, -2.24,  1.45],
       [ 1.2 , -0.34,  1.54, -0.39]])
print(node.ask_question(test_x)) # should be [True True]

0
0
[ True  True]


Now, using the Node class, we will implement the decision tree. Like scikit-learn's interface, we will have fit() and predict() methods.

/

Maintenant, en utilisant la classe Node, nous allons implémenter l'arbre de décision. Comme avec l'interface de scikit-learn, nous aurons les méthodes fit() et predict().

In [0]:
class DecisionTreeClassifier():
    def __init__(self, max_depth=1):
        self.max_depth = max_depth
    
    def create_node(self, x_subset, y_subset, depth):
        # Recursive function
        node = Node()
        
        majority_class = get_majority_class(y_subset)
        majority_class_count = (y_subset == majority_class).sum()
        perfectly_classified = majority_class_count == len(y_subset)
        
        if perfectly_classified or depth == self.max_depth:
            node.output_class = majority_class
            node.is_leaf = True
        else:
            node.find_best_question(x_subset,y_subset)
            node.is_leaf = False
            right_subset_rows = node.ask_question(x_subset) 
            left_subset_rows = np.invert(right_subset_rows)
            #
            # Recursion: create node.left_child and node.right_child here
            #
            # Récursion: créer node.left_child (enfant gauche du noeud) et
            # node.right_child (enfant droit du noeud) ici
            #
                
        return node
    
    def fit(self, x, y):
        self.root_node = self.create_node(x,y,depth=1)
    
    def predict(self, x):
        predictions = []
        
        for i in range(len(x)):
            current_node = self.root_node
            x_i = x[i].reshape(1,-1)
            done_descending_tree = False
            while not done_descending_tree:
                if current_node.is_leaf:
                    predictions.append(current_node.predict())
                    done_descending_tree = True

                else:
                    if current_node.ask_question(x_i):
                        current_node = current_node.right_child
                    else:
                        current_node = current_node.left_child

        return np.array(predictions)
        

Train a tree to test your implementation. (You can also try comparing with scikit-learn's tree implementation.)

/

Entrainez un arbre pour tester votre implémentation. (Vous pouvez aussi essayer de comparer avec l'implémentation d'arbre de scikit-learn.)

In [0]:
# import sklearn.tree
# tree = sklearn.tree.DecisionTreeClassifier(max_depth=10)
tree = DecisionTreeClassifier(max_depth=10)
tree.fit(iris_train_x, iris_train_y)
print(iris_test_y)
print(tree.predict(iris_test_x))
print("accuracy: ", (tree.predict(iris_test_x) == iris_test_y).mean() * 100, "%")

[1 0 0 1 2 0 2 1 2 2 1 2 0 2 1 1 0 2 2 2 0 0 0 0 2 1 2 1 1 0 2 2 1 0 1 0 1
 2 1 2 0 0 1 1 0 0 0 2 0 1 1 0 2 0 0 1 1 0 1 1 2 1 1 2 2 0 2 1 0 1 0 1 2 1
 0 1 0 1 0 1 1 0 0 2 2 1 0 1 2 1 0 0 0 2 2 1 2 1 0 1]


AttributeError: ignored

Train trees of increasing depth on the Iris dataset and plot their performance. What do you notice about the performance as the depth increases?

/

Entrainez des arbres de profondeur croissante sur le jeu de données Iris et tracez leurs performances. Que remarquez-vous sur les performances lorsque la profondeur augmente ?

In [0]:
depths = []
accuracies = []
for depth in range(1, 20):
    model = DecisionTreeClassifier(max_depth=depth)
    model.fit(iris_train_x, iris_train_y)
    accuracy = (model.predict(iris_test_x) == iris_test_y).mean()
    accuracies.append(accuracy)
    depths.append(depth)

print("Best performance: ", max(accuracies))
plt.xlabel('Tree depth')
plt.ylabel('Performance on test set')
plt.plot(depths, accuracies)

AttributeError: ignored

## Random forest

Next, we will implement the random forest model using our decision tree. 

A random forest is an ensemble of decision trees, where each tree is trained by randomizing the dataset. The standard method for randomizing the dataset is feature bagging, where a random subset of features is selected for each tree, but other methods are possible.

We will explore two ways of randomizing the dataset: 1) feature bagging, and 2) example bagging, i.e. training each classifier on a random subset of the dataset.

/

## Forêt aléatoire
Ensuite, nous allons implémenter une forêt aléatoire en utilisant notre arbre de décision.

Une forêt aléatoire est un ensemble d'arbres de décision, où chaque arbre est entrainé sur une variation aléatoire du jeu de données. La méthode standard pour obtenir ces variations est le feature bagging (ensachage d'attributs), où un sous-ensemble aléatoire des attributs est choisi pour chaque arbre, mais d'autres méthodes sont possibles.

Nous allons explorer deux façon d'obtenir des variations aléatoires du jeu de données: 1) feature bagging (ensachage d'attributs), et 2) example bagging (ensachage d'exemples), i.e. entrainer chaque classifieur sur un sous-ensemble aléatoire du jeu de données.

In [0]:
import math

class RandomForestClassifier():
    def __init__(self, n_estimators=2, max_depth=5, bootstrap_fraction=0.5, features_fraction=0.5):
        self.max_depth = max_depth
        self.n_estimators = n_estimators
        self.bootstrap_fraction = bootstrap_fraction
        self.features_fraction= features_fraction
        self.estimators = []
        
    def fit(self, x, y):
        num_rows = math.ceil(self.bootstrap_fraction * x.shape[0])
        num_cols = math.ceil(self.features_fraction * x.shape[1])
        for _ in range(self.n_estimators):
            rows_idx = np.random.choice(x.shape[0], size=num_rows)
            cols_idx = np.random.choice(x.shape[1], size=num_cols, replace=False)
            # 
            # Create noisy subsets x_subset, y_subset here
            # 
            # Créer ici les sous-ensembles aléatoires x_subset, y_subset
            # 
            x_subset, y_subset = [], []
            tree = DecisionTreeClassifier(max_depth=self.max_depth)
            tree.fit(x_subset, y_subset)
            self.estimators.append((tree,cols_idx))
        
    def predict(self, x):
        #
        # Predict output using all estimators here
        # 
        # Faire une prédiction se basant sur tout les estimateurs ici
        # 
        pass


Now let's look at the influence of these hyperparameters. First, let's plot the performance as a function of the number of trees when using feature bagging only. Next, let's try example bagging only. Then let's put them together.

/

Maintenant regardons l'influence de ces hyperparamètres. D'abord, traçons les performances en fonction du nombre d'arbres en utilisant uniquement le feature bagging (ensachage d'attributs). Ensuite, essayons l'example bagging (ensachage d'exemples). Finalement, combinons-les ensemble.



In [0]:
def validation_curve(bootstrap_fraction, features_fraction):
  tree_counts = []
  accuracies = []
  for tree_count in range(1, 20):
      model = RandomForestClassifier(n_estimators=tree_count,
                                    max_depth=3,
                                    bootstrap_fraction=bootstrap_fraction,
                                    features_fraction=features_fraction)
      model.fit(iris_train_x, iris_train_y)
      accuracy = (model.predict(iris_test_x) == iris_test_y).mean()
      accuracies.append(accuracy)
      tree_counts.append(tree_count)

  print("Best performance: ", max(accuracies))
  plt.xlabel('n_estimators')
  plt.ylabel('Performance on test set')
  plt.plot(tree_counts, accuracies)
  plt.show()

In [0]:
validation_curve(0.5,1)

AttributeError: ignored

In [0]:
validation_curve(1, 0.5)

AttributeError: ignored

In [0]:
validation_curve(.5,.5)